 # Opis zadania
 W tym notatniku stworzymy system analityczno-raportowy, mający wspomagać przyszłe decyzje biznesowe. Dzięki przeprowadzonej wcześniej analizie eksploracyjnej danych, w trakcie spotkania prezentującego wykonaną pracę postanowiono w pierwszym kroku stworzyć następujące wizualizacje danych:
 - Wyświetlenie TOP 10 tras samolotów pod względem najniższego odsetka opóźnień w formie tabeli,
 - Wyświetlenie porównania roku 2019 oraz 2020 w formie wykresu słupkowego:
     - miesiąc do miesiąca,
     - dzień tygodnia do dnia tygodnia,
 - Wyświetlenie danych dzień po dniu w formie szeregu czasowego.

Raport powinien składać się z trzech stron - `TOP report`, `Comparision` i `Day by day reliability`.

Pracując na podstawie wcześniej zdefiniowanych widoków, nie musimy się tutaj martwić o przetwarzanie oraz procesowanie danych. Co więcej, wszystkie niuanse techniczne w postaci liczby wierszy, wykonywanych złączeń, filtracji są przeniesione na bazę danych. Z technicznego punktu widzenia, pomiędzy serwisem a bazą danych dochodzi do przesyłania mniejszej ilości danych.

Dopuszczamy jednak pewne aktualizacje co do ich struktury przykładowo poprzez _pivotowanie_, czyli obranie i doprowadzanie do postaci tej znanej z tabel przestawnych.

> **Uwaga:**  
> Przy pracy nad poprawkami w dashboardzie, pamiętaj, że aby odświeżyć stronę po wprowadzonych zmianach, należy **całość** uruchomić ponownie.

W tej części projektu końcowego wcielasz się ponownie w rolę BI Developera, który ma za zadanie stworzyć dashboard zgodny z wytycznymi biznesowymi dostarczonymi przez klienta. Pamiętaj, że osoba na tym stanowisku często ma kontakt z biznesem więc musi umieć przekazać informację o danych, które zawarła na dashboardzie by te zaangażowały odbiorców.


 Tutaj zaimportuj potrzebne biblioteki

In [277]:
pip install dash

Note: you may need to restart the kernel to use updated packages.


In [278]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import psycopg2
from sqlalchemy import create_engine
import dash
from dash import html, dcc
from dash.dependencies import Input, Output
import plotly.express as px
import plotly.graph_objects as go
import dash_core_components as dcc
from dash import html
from dash_table import DataTable
from dash.exceptions import PreventUpdate
import threading

 ## Konfiguracja połączenia
 Tutaj uzupełnij konfigurację połączenia

In [279]:
username = 'postgres'
password = 'NoScar11'

host = 'localhost'
database = 'airlines'
port = 5432

 Tutaj zdefiniuj zmienną `con` oraz `cursor`

In [280]:
con = psycopg2.connect(host=host, user=username, password=password, dbname=database)
con.autocommit = True  
cursor = con.cursor()

 # Stworzenie tabeli dla `TOP 10 routes`
 W tym miejscu przygotujemy tabelkę oraz komponent, który zostanie następnie umieszczony w raporcie. Tabela powinna się składać z następujących kolumn:
 - Origin,
 - Destination,
 - Year,
 - Reliability,
 - Rank.

Wartości procentowe zaś powinny być postaci np. 13.87%.

Z tego względu konieczna będzie zmiana nazw kolumn dla wyników raportowania.

Wskazówki:
 - `Python - analiza danych > Dzień 5 - Pandas > Obróbka danych - część 2 > apply`,
 - [How to format percentage in python](https://stackoverflow.com/questions/5306756/how-to-print-a-percentage-value-in-python).

 Tutaj zaczytaj dane do ramki danych `top_routes_df` z widoku `reporting.top_reliability_roads`
 > Jeśli pojawi się komunikat `UserWarning: pandas only support SQLAlchemy`, z naszego punktu widzenia, możemy go śmiało zignorować.

In [281]:
top_routes_df = pd.read_sql("SELECT * FROM reporting.top_reliability_roads", con)

/var/folders/hp/4lm9kn39221_wddvkzwmdcf80000gn/T/ipykernel_14596/2965344040.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [282]:
top_routes_df.head()

,origin_airport_id,origin_airport_name,dest_airport_id,dest_airport_name,year,cnt,reliability,nb
0,13930,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",12953,"LAGUARDIA AIRPORT, NY US",2019,14219,0.41,1
1,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",12478,"LAGUARDIA AIRPORT, NY US",2019,12797,0.39,2
2,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",12889,"MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,11628,0.38,3
3,12889,"MCCARRAN INTERNATIONAL AIRPORT, NV US",12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,11616,0.37,4
4,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",14771,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,14096,0.37,4


 Tutaj dokonaj aktualizacji formatu liczbowego dla kolumny `reliability`

In [283]:
top_routes_df['reliability'] = top_routes_df['reliability'].apply(lambda x: '{:.2%}'.format(x))

In [284]:
top_routes_df['reliability'] 

0    41.00%
1    39.00%
2    38.00%
3    37.00%
4    37.00%
5    36.00%
6    34.00%
7    27.00%
8    27.00%
9    25.00%
Name: reliability, dtype: object

 Tutaj odpowiednio dokonaj zmian nazewnictwa kolumn

In [285]:
top_routes_df.columns

Index(['origin_airport_id', 'origin_airport_name', 'dest_airport_id',
       'dest_airport_name', 'year', 'cnt', 'reliability', 'nb'],
      dtype='object')

In [286]:
top_routes_df = top_routes_df.rename(columns={
    'origin_airport_name': 'Origin',
    'dest_airport_name': 'Destination',
    'year': 'Year',
    'reliability': 'Reliability',
    'nb': 'Rank'
})

In [287]:
top_routes_df.head()

,origin_airport_id,Origin,dest_airport_id,Destination,Year,cnt,Reliability,Rank
0,13930,"CHICAGO OHARE INTERNATIONAL AIRPORT, IL US",12953,"LAGUARDIA AIRPORT, NY US",2019,14219,41.00%,1
1,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",12478,"LAGUARDIA AIRPORT, NY US",2019,12797,39.00%,2
2,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",12889,"MCCARRAN INTERNATIONAL AIRPORT, NV US",2019,11628,38.00%,3
3,12889,"MCCARRAN INTERNATIONAL AIRPORT, NV US",12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",2019,11616,37.00%,4
4,12892,"LOS ANGELES INTERNATIONAL AIRPORT, CA US",14771,"SAN FRANCISCO INTERNATIONAL AIRPORT, CA US",2019,14096,37.00%,4


 # Porównanie rok do roku (YoY) 2019 vs. 2020
 W tym miejscu stworzymy wykres oraz komponent, który zostanie następnie umieszczony w raporcie. Wykres powinien przedstawiać porównanie lat 2019 oraz 2020 po miesiącach względem zmiennej `reliability`.

 > Na wykresie chcemy również przedstawić dane, które nie mają porównania tj. od kwietnia do grudnia 2019.

W tym celu wykonamy poniższe zadania:
 - Przekształcimy tabelkę do formy tabeli przestawnej
 - Stworzymy wykres słupkowy porównujący dane rok do roku
 - Opakujemy stworzony wykres w komponent `Dash`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 1 > Budowanie wykresów plotly`,
 - `Python - analiza danych > Dzień 6 - Pandas c.d > Pivot`.

 Tutaj pobierz z widoku `reporting.year_to_year_comparision` dane do ramki `yoy_comparision_df`

In [288]:
yoy_comparision_df = pd.read_sql("SELECT * FROM reporting.year_to_year_comparision", con)

/var/folders/hp/4lm9kn39221_wddvkzwmdcf80000gn/T/ipykernel_14596/3793223248.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [289]:
yoy_comparision_df.head()

,year,month,flights_amount,reliability
0,2019,1,83876,0.35
1,2019,2,76834,0.40
2,2019,3,88547,0.36
3,2019,4,86319,0.35
4,2019,5,90875,0.38


 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `yoy_comparision_to_plot_df`

In [290]:
yoy_comparision_to_plot_df = yoy_comparision_df.pivot(index='month', columns='year', values='reliability')


In [291]:
yoy_comparision_to_plot_df.head()

year,2019
month,
1,0.35
2,0.40
3,0.36
4,0.35
5,0.38


 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `yoy_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [292]:
yoy_comparision_fig = go.Figure()

for year in yoy_comparision_to_plot_df.columns:
    yoy_comparision_fig.add_trace(go.Bar(
        x=yoy_comparision_to_plot_df.index,
        y=yoy_comparision_to_plot_df[year],
        name=str(year),
        text=yoy_comparision_to_plot_df[year].round(2),
        textposition='auto',
    ))

yoy_comparision_fig.update_layout(barmode='group', xaxis_title='Month', yaxis_title='Reliability', title='Year-to-Year Comparison (2019 vs. 2020)')

yoy_comparision_fig.show()

 # Porównanie dzień tygodnia do dnia tygodnia (WKoWK) 2019 vs. 2020
 Wykonaj analogiczne do poprzedniego kroku, używając jednak w tym momencie danych z widoku `reporting.day_to_day_comparision`

 Tutaj pobierz z widoku `reporting.day_to_day_comparision` dane do ramki `day_to_day_comparision_df`

In [293]:
day_to_day_comparision_df = pd.read_sql("SELECT * FROM reporting.day_to_day_comparision", con)

/var/folders/hp/4lm9kn39221_wddvkzwmdcf80000gn/T/ipykernel_14596/1452960934.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [294]:
day_to_day_comparision_df.head()

,year,day_of_week,flights_amount
0,2019,1,158693
1,2019,2,156704
2,2019,3,154610
3,2019,4,157352
4,2019,5,159632


In [295]:
day_to_day_comparision_df.columns

Index(['year', 'day_of_week', 'flights_amount'], dtype='object')

 Tutaj odpowiednio przekształć ramkę, do formy oczekiwanej przez wykres słupkowy. Wyniki zapisz do zmiennej `day_to_day_comparision_to_plot_df`

In [296]:
day_to_day_comparision_to_plot_df = day_to_day_comparision_df.pivot(index='day_of_week', columns='year', values='flights_amount')

In [297]:
day_to_day_comparision_to_plot_df.head()

year,2019
day_of_week,
1,158693
2,156704
3,154610
4,157352
5,159632


 Tutaj stwórz odpowiedni wykres, zapisz do na później pod nazwą `day_to_day_comparision_fig`. Pamiętaj o dobrych praktykach - niech wykres posiada wszystkie opisy oraz opisowe etykiety osi.

In [298]:
day_to_day_comparision_fig = px.bar(day_to_day_comparision_df, 
                                     x='day_of_week', 
                                     y='flights_amount', 
                                     color='flights_amount', 
                                     barmode='group', 
                                     labels={'flights_amount': 'Flights Amount', 'day_of_week': 'Day of Week', 'year': 'Year'},
                                     title='Week-to-Week Comparison (2019 vs. 2020)')


day_to_day_comparision_fig.show()

In [299]:
# podział danych na dwie serie (dla lat 2019 i 2020)
df_2019 = day_to_day_comparision_df[day_to_day_comparision_df['year'] == 2019]
df_2020 = day_to_day_comparision_df[day_to_day_comparision_df['year'] == 2020]


day_to_day_comparision_fig = go.Figure()


day_to_day_comparision_fig.add_trace(go.Bar(
    x=df_2019['day_of_week'],
    y=df_2019['flights_amount'],
    name='2019',
    marker_color='blue'  
))


day_to_day_comparision_fig.add_trace(go.Bar(
    x=df_2020['day_of_week'],
    y=df_2020['flights_amount'],
    name='2020',
    marker_color='orange'  
))


day_to_day_comparision_fig.update_layout(barmode='group', xaxis_title='Day of Week', yaxis_title='Flights Amount', title='Week-to-Week Comparison (2019 vs. 2020)')


day_to_day_comparision_fig.show()

 # Stworzenie szeregu czasowego
 W tym miejscu stworzymy wykres w formie szeregu czasowego, który zostanie umieszcony w raporcie. Wykres powinien przedstawiać dane w formie szeregu czasowego (dzień po dniu) dla lat 2019 oraz 2020. Dla możliwości identyfikacji poszczególnych lat, zostaliśmy dodatkowo poproszeni o nadanie innego koloru dla roku 2019 oraz 2020.

 > Pamiętaj o stworzeniu wykresu zgodnie z dobrymi praktykami.

 W tym miejscu pobierz dane do ramki `day_by_day_reliability_df`, z widoku `reporting.day_by_day_reliability`.

In [300]:
day_by_day_reliability_df = pd.read_sql("SELECT * FROM reporting.day_by_day_reliability", con)

/var/folders/hp/4lm9kn39221_wddvkzwmdcf80000gn/T/ipykernel_14596/3084814644.py:1: UserWarning:

pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.



In [301]:
day_by_day_reliability_df.head()

,date,reliability
0,2019-01-01,0.35
1,2019-01-02,0.32
2,2019-01-03,0.33
3,2019-01-04,0.39
4,2019-01-05,0.35


In [302]:
day_by_day_reliability_df.columns

Index(['date', 'reliability'], dtype='object')

 Tutaj stwórz wykres liniowy na podstawie pobranych danych. Wynik zapisz do zmiennej `day_by_day_reliability_fig`.

In [303]:
day_by_day_reliability_df['year'] = pd.to_datetime(day_by_day_reliability_df['date']).dt.year


day_by_day_reliability_fig = go.Figure()


day_by_day_reliability_fig.add_trace(go.Scatter(
    x=day_by_day_reliability_df['date'][:365],
    y=day_by_day_reliability_df['reliability'][:365],
    mode='lines',
    name='2019',
    line=dict(color='blue')  
))


day_by_day_reliability_fig.add_trace(go.Scatter(
    x=day_by_day_reliability_df['date'][365:],
    y=day_by_day_reliability_df['reliability'][365:],
    mode='lines',
    name='2020',
    line=dict(color='orange')  
))

day_by_day_reliability_fig.update_layout(xaxis_title='Date', yaxis_title='Reliability', title='Day-by-Day Reliability (2019 vs. 2020)')


day_by_day_reliability_fig.show()


 # Stworzenie layoutów poszczególnych stron

 ## Stworzenie layout dla widoku `TOP 10 routes`
 W tym momencie przekonwertujemy zmienną `top_routes_df` w postać `Dash.DataTable`. Dodatkowo, dla czytelności, dodamy nagłówek opisujący co tabela przedstawia.

 Nagłówek wystylizuj używając poniższych wymagań:
 - tag: `H3`,
 - czcionka: `verdana`,
 - kolor: '#4444`,
 - tekst do wyświetlenia: `TOP 10 reliability routes in 2019 and 2020`

Wskazówki:
 - `Wizualizacja danych > Dzień 3 > Dash`,
 - `Wizualizacja danych > Dzień 3 > Dash - datatable`,
 - Dokmentacja metody `Pandas` - [to_dict](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.to_dict.html).

 Tutaj przygotuj komponent `Dash` `DataTable`, zapisując wynik do zmiennej `top_routes_table`

In [304]:
top_routes_table = DataTable(
    id='top-routes-table',
    columns=[{'name': col, 'id': col} for col in top_routes_df.columns],
    data=top_routes_df.to_dict('records'),
    style_table={'height': '400px', 'overflowY': 'auto'},
)

 Tutaj przygotuj nagłówek dla tabeli, zapisz go pod nazwą `top_routes_page_title`

In [305]:
top_routes_page_title = html.H3(
    children='TOP 10 reliability routes in 2019 and 2020',
    style={'fontFamily': 'verdana', 'color': '#4444'},
)

 Tutaj przygotuj layout dla widoku `TOP 10 routes`. Konfigurację zapisz do zmiennej `top_routes_layout`

In [306]:
top_routes_layout = html.Div([
    top_routes_page_title,
    top_routes_table,
])

 ## Stworzenie layout dla widoku `Comparision`
 W tym momencie opakujemy zmienne `yoy_comparision_fig` oraz `day_to_day_comparision_fig` do komponentów `Dash`, a następnie stworzymy odpowiedni `layout` dla strony `comparision`. Wyniki zapiszemy do zmiennej `comparision_layout`.

 Tutaj opakuj `yoy_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `yoy_comparision_component`.

In [307]:
yoy_comparision_component = dcc.Graph(
    id='yoy-comparision-graph',
    figure=yoy_comparision_fig,
)

 Tutaj opakuj `day_to_day_comparision_fig` w odpowiedni komponent `Dash`. Wynik zapisz do zmiennej `day_to_day_comparision_component`.

In [308]:
day_to_day_comparision_component = dcc.Graph(
    id='day-to-day-comparision-graph',
    figure=day_to_day_comparision_fig,
)


 Tutaj stwórz odpowiedni `layout` dla widoku `comparision`. Wynik zapisz do zmiennej `comparision_layout`.

In [309]:
comparision_layout = html.Div([
    html.H3('Year-to-Year and Week-to-Week Comparison'),
    html.Div([
        html.Div([
            html.H4('Year-to-Year Comparison (2019 vs. 2020)'),
            yoy_comparision_component,
        ], className='six columns'),
        html.Div([
            html.H4('Week-to-Week Comparison (2019 vs. 2020)'),
            day_to_day_comparision_component,
        ], className='six columns'),
    ], className='row'),
])


 Tutaj stwórz odpowiedni `layout` dla widoku `day by day`. Wynik zapisz do zmiennej `day_by_day_layout`.

In [310]:
day_by_day_reliability_component = dcc.Graph(
    id='day-by-day-reliability-graph',
    figure=day_by_day_reliability_fig,
)


day_by_day_layout = html.Div([
    html.H3('Day-by-Day Reliability'),
    html.Div([
        html.Div([
            html.H4('Day-by-Day Reliability'),
            day_by_day_reliability_component,
        ], className='twelve columns'),
    ], className='row'),
])


 # Konfigurowanie aplikacji
 Posiadamy już wszystkie niezbędne komponenty do stworzenia oraz uruchomienia aplikacji. Naszym zadaniem w tym miejscu będzie odpowiednie jej skonfigurowanie, aby obsługiwała więcej niż jedną stronę.

 Chcemy, aby możliwe było poruszanie się pomiędzy poniższymi stronami:
 - `TOP report` - strona domyślna,
 - `Comparision` - strona dostępna pod adresem: `comparision_reporting`,
 - `Day by day reporting` - strona dostępna pod adresem: `day_by_day_reporting`

Wskazówki:
 - `Wizualizacja danych > Dzień 4 > Callback context`,
 - `Wizualizacja danych > Dzień 4 > Aplikacja multipage`.
 - Używając komponentu [html.Button](https://community.plotly.com/t/button-with-link/11809), możesz łatwo stworzyć ładniejsze linki do poszczególnych stron.

 Tutaj przygotuj layout aplikacji

In [354]:
app = dash.Dash()
app.layout = html.Div([
    dcc.Location(id='url', refresh=False),
    dcc.Tabs(
        id='tabs',
        value='top-report',
        children=[
            dcc.Tab(label='TOP Report', value='top-report'),
            dcc.Tab(label='Comparison', value='comparison'),
            dcc.Tab(label='Day by Day Reporting', value='day-by-day')
        ]
    ),
    html.Div(id='page-content')
])

 Tutaj stwórz procedurę odpowiadającą za nawigację pomiędzy poszczególnymi stronami

In [358]:
@app.callback(
    Output('page-content', 'children'),
    [Input('tabs', 'value')]
)
def display_page(tab):
    if tab == 'top-report':
        return top_routes_layout
    elif tab == '/comparison_reporting':
        return comparision_layout
    elif tab == '/day-by-day_reporting':
        return day_by_day_layout
    else:
        return top_routes_layout  
    


 Tutaj uruchom aplikację

In [357]:
app.run()

[2023-12-17 23:54:49,585] ERROR in app: Exception on /_dash-component-suites/dash/deps/react@16.v2_14_0m1697880497.14.0.js [GET]
Traceback (most recent call last):
  File "/Users/aleksandraobrzut/anaconda3/lib/python3.10/site-packages/flask/app.py", line 2525, in wsgi_app
    response = self.full_dispatch_request()
  File "/Users/aleksandraobrzut/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1822, in full_dispatch_request
    rv = self.handle_user_exception(e)
  File "/Users/aleksandraobrzut/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1820, in full_dispatch_request
    rv = self.dispatch_request()
  File "/Users/aleksandraobrzut/anaconda3/lib/python3.10/site-packages/flask/app.py", line 1796, in dispatch_request
    return self.ensure_sync(self.view_functions[rule.endpoint])(**view_args)
  File "/Users/aleksandraobrzut/anaconda3/lib/python3.10/site-packages/dash/dash.py", line 947, in serve_component_suites
    _validate.validate_js_path(self.registered_path

 # Podsumowanie
 W tym notatniku stworzyliśmy zgodnie z wymaganiami raport do dyspozycji osób decyzyjnych. Kod źródłowy może w tym momencie zostać przekazany dalej, do działu IT, który następnie wdroży rozwiązanie na serwer dostępny dla każdej zainteresowanej osoby. W praktyce oznacza to koniec naszej pracy nad tym zadaniem. Choć warto dodać, że często po udostępnieniu raportu pojawiają się dodatkowe wymagania oraz komentarze ze względu na informacje, które są tam zawarte.

 W kolejnym notatniku podsumujemy sobie cały warsztat.